In [ ]:
import pandas as pd
import numpy as np

### Cleaning the data

In [ ]:
df = pd.read_csv('Financial Distress.csv')
print(df.isnull().sum())
df = df.groupby('Company').mean(numeric_only=True).reset_index()
df = df.drop(columns=['Time'])
print(df.head(5))
print(len(df))

### Vizualize tha data or sumn